In [100]:
def encode(rawStr):
    binary_data = ''.join(format(ord(char), '08b') for char in rawStr)
    

    while len(binary_data) % 5 != 0:
        binary_data += '0'
    
    index_char_map = '9876543210mnbvcxzasdfghjklpoiuyt'
    encoded = ''
    
    for i in range(0, len(binary_data), 5):
        five_bit = binary_data[i:i+5]
        index = int(five_bit, 2)
        encoded += index_char_map[index]
    

    remainder = len(rawStr) * 8 % 40
    if remainder == 0:
        encoded += ''
    elif remainder == 8:
        encoded += '++++++'
    elif remainder == 16:
        encoded += '++++'
    elif remainder == 24:
        encoded += '+++'
    elif remainder == 32:
        encoded += '+'
    
    return encoded

def decode(encodedStr):
    index_char_map = '9876543210mnbvcxzasdfghjklpoiuyt'
    binary_data = ''
    
    for char in encodedStr:
        if char in index_char_map:
            index = index_char_map.index(char)
            binary_data += format(index, '05b')
        elif char == '+':
            continue
        else:
            raise ValueError(f"Unexpected character '{char}' in encoded string.")

    decoded_chars = []
    for i in range(0, len(binary_data), 8):
        byte = binary_data[i:i+8]
        if len(byte) == 8:
            decoded_chars.append(chr(int(byte, 2)))
    
    return ''.join(decoded_chars)

# Example usage
rawStr = "foo"
encodedStr = "bljhy+++b0zj5+++"

encoded_result = encode(rawStr)
decoded_result = decode(encodedStr)

print(f"Encoded: {encoded_result}")
print(f"Decoded: {decoded_result}")


Encoded: bljhy+++
Decoded: foo10¹


In [6]:
def decode(encodedStr):
    index_char_map = '9876543210mnbvcxzasdfghjklpoiuyt'
    binary_data = ''
    notes = 0
    chars = 0
    note = len(encodedStr)
    print(note)
    while notes < note:
        char = encodedStr[notes]
        if char in index_char_map:
            index = index_char_map.index(char)
            binary_data += format(index, '05b')
        elif char == '+':
            while(notes < note and encodedStr[notes] == '+'):
                notes += 1
                chars += 1
            print(notes,chars)
            chars = 0
            continue
        else:
            raise ValueError(f"Unexpected character '{char}' in encoded string.")
        notes+=1


    decoded_chars = []
    for i in range(0, len(binary_data), 8):
        byte = binary_data[i:i+8]
        if len(byte) == 8:
            decoded_chars.append(chr(int(byte, 2)))
    
    return ''.join(decoded_chars)

# Example usage
rawStr = "foo"
encodedStr = "bljhy+++b0zj5+++"

decoded_result = decode(encodedStr)

print(f"Decoded: {decoded_result}")

16
8 3
16 3
Decoded: foo10¹


In [9]:
from itertools import combinations
from collections import Counter

def find_original_numbers(n, sums):
    # Sort the sums
    sums.sort()
    
    if n == 1:
        return "Impossible" if sums[0] != 0 else "0"
    
    total_pairs = len(sums)
    min_sum = sums[0]
    
    possible_numbers = []
    
    # Loop through sums to deduce possible original numbers
    for i in range(1, total_pairs):
        for j in range(i + 1, total_pairs):
            a0 = (min_sum + sums[i] - sums[j]) // 2
            a1 = (min_sum + sums[j] - sums[i]) // 2
            
            if (a0 + a1) != min_sum:
                continue
            
            original_numbers = [a0, a1]
            a_sum = a0 + a1
            
            # Deduce the next numbers
            for k in range(n - 2):
                next_sum = sums[2 + k] - a_sum
                original_numbers.append(next_sum)
                a_sum += next_sum
            
            # Calculate all possible pairwise sums from original_numbers
            calculated_sums = [a + b for a, b in combinations(original_numbers, 2)]
            
            if Counter(calculated_sums) == Counter(sums):
                possible_numbers.append(sorted(original_numbers))
    
    if not possible_numbers:
        return "Impossible"
    
    # Return the first valid solution
    return ' '.join(map(str, possible_numbers[0]))

# Test cases
print(find_original_numbers(3, [1269, 1160, 1663]))  # Expected: "383 777 886"
print(find_original_numbers(3, [1, 1, 1]))          # Expected: "Impossible"
print(find_original_numbers(5, [226, 223, 225, 224, 227, 229, 228, 226, 225, 227]))  # Expected: "111 112 113 114 115"
print(find_original_numbers(5, [-1, 0, -1, -2, 1, 0, -1, 1, 0, -1]))  # Expected: "-1 -1 0 0 1"
print(find_original_numbers(5, [79950, 79936, 79942, 79962, 79954, 79972, 79960, 79968, 79924, 79932]))  # Expected: "39953 39971 39979 39983 39989"


Impossible
Impossible
Impossible
-1 -1 0 0 1
Impossible


In [21]:
def calculate(expression):
    def evaluate(ops, nums):
        right = nums.pop()
        left = nums.pop()
        op = ops.pop()
        if op == '+':
            nums.append(left + right)
        elif op == '-':
            nums.append(left - right)
        elif op == '*':
            nums.append(left * right)
        elif op == '/':
            nums.append(left // right) 

    def precedence(op):
        if op in ('+', '-'):
            return 1
        if op in ('*', '/'):
            return 2
        return 0

    ops = []
    nums = []
    i = 0

    while i < len(expression):
        if expression[i].isdigit():
            num = 0
            while i < len(expression) and expression[i].isdigit():
                num = num * 10 + int(expression[i])
                i += 1
            nums.append(num)
            continue
        elif expression[i] in "+-*/":
            while (ops and precedence(ops[-1]) >= precedence(expression[i])):
                evaluate(ops, nums)
            ops.append(expression[i])
        elif expression[i] == '(':
            ops.append(expression[i])
        elif expression[i] == ')':
            while ops and ops[-1] != '(':
                evaluate(ops, nums)
            ops.pop() 
        i += 1

    while ops:
        evaluate(ops, nums)

    return nums[0]

# 测试样例
print(calculate("1+1"))  # 输出: 2
print(calculate("3+4*5/(3+2)"))  # 输出: 7
print(calculate("4+2*5-2/1"))  # 输出: 12
print(calculate("(1+(4+5+2)-3)+(6+8)"))  # 输出: 23
print(calculate("2*(5+5*2)/3+(6+8*3)"))  # 输出: 40


2
7
12
23
40
